# 04_04_hdf5.ipynb - Scientific datasets with HDF5 

In [ ]:
import numpy as np
import scipy.interpolate
import matplotlib.pyplot as pp
import matplotlib.mlab as mlab

import h5py

In [ ]:
# if you don't have the hdf5 files from 04_02_web, 
# you can find them in the folder cache
h1file = h5py.File('H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'r')

In [ ]:
h1file

In [ ]:
h1file.visititems(print)

In [ ]:
h1file.keys()

In [ ]:
h1file['meta']['Description'][()]

In [ ]:
h1file['meta/UTCstart'][()]

In [ ]:
h1file['meta/GPSstart'][()]

In [ ]:
h1strain = h1file['strain/Strain']

In [ ]:
h1strain

In [ ]:
h1strain[:10]

In [ ]:
h1attrs = h1file['strain/Strain'].attrs

In [ ]:
h1attrs

In [ ]:
dict(h1attrs)

In [ ]:
gpstime = h1attrs['Xstart'] + h1attrs['Xspacing'] * np.arange(h1attrs['Npoints'])

In [ ]:
pp.plot(gpstime, h1strain)

In [ ]:
tevent = 1126259462.4

In [ ]:
around = (gpstime > tevent - 0.1) & (gpstime < tevent + 0.1)

In [ ]:
pp.plot(gpstime[around] - tevent, h1strain[around])

In [ ]:
psd, freqs = mlab.psd(h1strain,
                      NFFT=4096*4,  # use four seconds of data to compute the spectrum
                      Fs=4096)      # this is the sampling rate of the data in Hz

pp.loglog(freqs, np.sqrt(psd))      # plot the sqrt of spectrum (power spectral density)

pp.xlabel('f [Hz]'); pp.ylabel('PSD [1/sqrt(Hz)]')
pp.axis(xmin=5, xmax=1024, ymin=1e-24, ymax=1e-18);

In [ ]:
def whiten(strain, sampling=4096, bandpass=[20,350]):
    # compute the spectrum
    psd, psd_freqs = mlab.psd(strain, NFFT=sampling*4, Fs=sampling)

    # max out the spectrum outside the bandpass
    psd[(psd_freqs < bandpass[0]) | (psd_freqs > bandpass[1])] = np.max(psd)
    
    dt = 1/sampling

    # compute the real FFT and the corresponding frequencies
    strain_fft = np.fft.rfft(strain)
    fft_freqs = np.fft.rfftfreq(len(strain), dt)

    # interpolate the spectrum to the FFT frequencies
    psd_fft = scipy.interpolate.interp1d(psd_freqs, psd)(fft_freqs)
    
    # whiten and transform back to real space
    whitened = np.fft.irfft(strain_fft / np.sqrt(psd_fft) * np.sqrt(2*dt))
    
    return whitened

In [ ]:
pp.figure(figsize=(12,3))
pp.plot(gpstime[around] - tevent, whiten(h1strain)[around])

In [ ]:
l1file = h5py.File('L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'r')

In [ ]:
l1strain = l1file['strain/Strain']

In [ ]:
pp.figure(figsize=(12,3))
pp.plot(gpstime[around] - tevent, whiten(h1strain)[around])
pp.plot(gpstime[around] - tevent, whiten(l1strain)[around])

In [ ]:
pp.figure(figsize=(12,3))
pp.plot(gpstime[around] - tevent, whiten(h1strain)[around])
pp.plot(gpstime[around] - tevent + 0.007, -whiten(l1strain)[around])